## <small>
Copyright (c) 2017-21 Andrew Glassner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
</small>



# Deep Learning: A Visual Approach
## by Andrew Glassner, https://glassner.com
### Order: https://nostarch.com/deep-learning-visual-approach
### GitHub: https://github.com/blueberrymusic
------

### What's in this notebook

This notebook is provided as a “behind-the-scenes” look at code used to make some of the figures in this chapter. It is cleaned up a bit from the original code that I hacked together, and is only lightly commented. I wrote the code to be easy to interpret and understand, even for those who are new to Python. I tried never to be clever or even more efficient at the cost of being harder to understand. The code is in Python3, using the versions of libraries as of April 2021. 

This notebook may contain additional code to create models and images not in the book. That material is included here to demonstrate additional techniques.

Note that I've included the output cells in this saved notebook, but Jupyter doesn't save the variables or data that were used to generate them. To recreate any cell's output, evaluate all the cells from the start up to that cell. A convenient way to experiment is to first choose "Restart & Run All" from the Kernel menu, so that everything's been defined and is up to date. Then you can experiment using the variables, data, functions, and other stuff defined in this notebook.

## Chapter 19: RNNs - Notebook 3: Holmes

This notebook is a bit more casual than most of the notebooks in this repo,
as it's only meant to do a single specific thing. There's no organization
into useful functions and subroutines - it's just one cell after another,
computing things in sequence! Feel free to re-organize it if you'd like to 
do more general-purpose text generation.

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

In [7]:
# Workaround for Keras issues on Mac computers (you can comment this
# out if you're not on a Mac, or not having problems)
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
window_length = 40
window_step = 3
number_of_epochs = 100
generated_text_length = 1000
batch_size = 100
test_input_file = 'input_data/test-holmes.txt'
input_file = 'input_data/holmes.txt'
output_file =  'holmes-by-char.txt'

file = open(input_file, 'r', encoding='utf-8') 
text = file.read()
file.close()
#text = text.lower()
# replace newlines with blanks, and double blanks with singles
text = text.replace('\n',' ') 
text = text.replace('  ', ' ')
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# make overlapping sequences of window_length characters
sentences = []
next_chars = []
for i in range(0, len(text)-window_length, window_step):
    sentences.append(text[i: i + window_length])
    next_chars.append(text[i + window_length])
print('number of sequences of ',window_length,':', len(sentences))

# Turn inputs and targets into one-hot versions
X = np.zeros((len(sentences), window_length, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# build the model
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(window_length, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def print_string(out_str=''):
    print(out_str, end='')
    File_writer.write(out_str)
    
File_writer = open(output_file, 'w')

# train the model, output generated text after each iteration
for iteration in range(number_of_epochs):
    print_string('--------------------------------------------------\n')
    print_string('Iteration '+str(iteration)+'\n')
    history = model.fit(X, y, batch_size=batch_size, epochs=1)
    print_string('  Loss from iteration '+str(iteration)+' = '+str(history.history['loss'])+'\n')
    start_index = random.randint(0, len(text) - window_length - 1)

    for diversity in [0.5, 1.0, 1.5, 2.0]:
        print_string('\n----- diversity: '+str(diversity)+'\n')

        generated = ''
        sentence = text[start_index: start_index + window_length]
        generated += sentence
        print_string('----- Generating with seed: <'+sentence+'>\n')
        
        for i in range(generated_text_length):
            x = np.zeros((1, window_length, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            #print_string(next_char)
            #sys.stdout.write(next_char)
            #sys.stdout.flush()
        print_string(generated+'\n\n')
        #print_string('\n\n')
        File_writer.flush()
print_string('\n')
File_writer.close()

corpus length: 1637265
total chars: 89
number of sequences of  40 : 545742
Build model...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 40, 128)        │       111,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 89)             │        11,481 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 254,681 (994.85 KB)

 Trainable params: 254,681 (994.85 KB)

 Non-trainable params: 0 (0.00 B)

--------------------------------------------------
Iteration 0
5458/5458 ━━━━━━━━━━━━━━━━━━━━ 420s 76ms/step - loss: 1.7749
  Loss from iteration 0 = [1.7749133110046387]

----- diversity: 0.5
----- Generating with seed: <lf fallen in love with the lady. You cou>
lf fallen in love with the lady. You could say the complete and my marked of the matter, and the loor and he was very read little in the man a smeap which which would have not one of the room, and he alserve to two read of the orine read come of the man who has been window come to me my complete of the man which and the long of the gray in the mind lest of the room were a great and would have in a parter be a fese of the letters and myself a scast to me me a son to see you the Tread of the same of the man with him them who was possible of the room and the poss of the stablers. The strange examine. Holmes show the strenge of the state windows of the man and had been to as he was examined with him home to the other way of the ev